In [1]:
# Import required libraries for image processing, deep learning, and visualization
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Set up GPU device and check PyTorch/CUDA versions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

Torch: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [3]:
# Configure hyperparameters and data paths for model training
IMAGE_DIR  = r"C:\Daily drive\College\sem 6\Main Project DL\data\xbd\train\images"
TARGET_DIR = r"C:\Daily drive\College\sem 6\Main Project DL\data\xbd\train\targets"

IMG_SIZE = 256
BATCH_SIZE = 8
EPOCHS = 6              # reduced (safe & sufficient)
NUM_CLASSES = 5
MAX_IMAGES = 1200       # 🔥 reduced dataset (KEY FIX)

In [4]:
# Verify that data files exist and explore sample images and target masks
print("Sample image files:")
print(os.listdir(IMAGE_DIR)[:5])

print("\nSample target files:")
print(os.listdir(TARGET_DIR)[:5])

Sample image files:
['guatemala-volcano_00000000_post_disaster.png', 'guatemala-volcano_00000000_pre_disaster.png', 'guatemala-volcano_00000001_post_disaster.png', 'guatemala-volcano_00000001_pre_disaster.png', 'guatemala-volcano_00000002_post_disaster.png']

Sample target files:
['guatemala-volcano_00000000_post_disaster_target.png', 'guatemala-volcano_00000000_pre_disaster_target.png', 'guatemala-volcano_00000001_post_disaster_target.png', 'guatemala-volcano_00000001_pre_disaster_target.png', 'guatemala-volcano_00000002_post_disaster_target.png']


In [5]:
# Define custom Dataset class to load and preprocess building damage images and target masks
class XBDDataset(Dataset):
    def __init__(self, image_dir, target_dir, max_images):
        self.image_dir = image_dir
        self.target_dir = target_dir

        all_images = sorted(os.listdir(image_dir))
        self.pairs = []

        for img_name in all_images:
            # Only use post-disaster images (recommended)
            if "post_disaster" not in img_name:
                continue

            mask_name = img_name.replace(".png", "_target.png")
            mask_path = os.path.join(target_dir, mask_name)

            if os.path.exists(mask_path):
                self.pairs.append((img_name, mask_name))

            if len(self.pairs) >= max_images:
                break

        if len(self.pairs) == 0:
            raise RuntimeError("❌ No matching image-mask pairs found.")

        print(f"✔ Matched image-mask pairs: {len(self.pairs)}")

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_name, mask_name = self.pairs[idx]

        img = cv2.imread(os.path.join(self.image_dir, img_name))
        mask = cv2.imread(
            os.path.join(self.target_dir, mask_name),
            cv2.IMREAD_GRAYSCALE
        )

        # Safety fallback
        if img is None or mask is None:
            new_idx = np.random.randint(0, len(self.pairs))
            return self.__getitem__(new_idx)

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)

        mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_NEAREST)
        mask = torch.tensor(mask, dtype=torch.long)

        return img, mask

In [6]:
# Create training dataset and dataloader for batch processing
train_dataset = XBDDataset(
    IMAGE_DIR,
    TARGET_DIR,
    MAX_IMAGES
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,   # Windows-safe
    pin_memory=True
)

print("Training samples:", len(train_dataset))

✔ Matched image-mask pairs: 1200
Training samples: 1200


In [7]:
# Define UNet architecture for semantic segmentation of building damage
class UNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1), nn.ReLU()
        )

        self.enc2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU()
        )

        self.pool = nn.MaxPool2d(2)

        self.bottleneck = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU()
        )

        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = nn.Conv2d(128, 64, 3, padding=1)

        self.up2 = nn.ConvTranspose2d(64, 32, 2, stride=2)
        self.dec2 = nn.Conv2d(64, 32, 3, padding=1)

        self.out = nn.Conv2d(32, num_classes, 1)

    def forward(self, x):
        c1 = self.enc1(x)
        c2 = self.enc2(self.pool(c1))

        b = self.bottleneck(self.pool(c2))

        u1 = self.up1(b)
        d1 = self.dec1(torch.cat([u1, c2], dim=1))

        u2 = self.up2(d1)
        d2 = self.dec2(torch.cat([u2, c1], dim=1))

        return self.out(d2)

In [8]:
# Initialize model, loss function, optimizer, and train the model for specified epochs
model = UNet(NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs = imgs.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {epoch_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "unet_damage_model.pth")
print("✅ Model saved successfully")

Epoch 1/6: 100%|██████████| 150/150 [00:59<00:00,  2.51it/s]


Epoch 1 Loss: 0.5616


Epoch 2/6: 100%|██████████| 150/150 [01:00<00:00,  2.50it/s]


Epoch 2 Loss: 0.2789


Epoch 3/6: 100%|██████████| 150/150 [00:53<00:00,  2.80it/s]


...

Epoch 4 Loss: 0.2521


Epoch 5/6: 100%|██████████| 150/150 [00:50<00:00,  2.96it/s]


Epoch 5 Loss: 0.2484


Epoch 6/6: 100%|██████████| 150/150 [00:54<00:00,  2.74it/s]

Epoch 6 Loss: 0.2429
✅ Model saved successfully


In [9]:
MODEL_PATH = r"c:\Daily drive\College\sem 6\Main Project DL\unet_damage_model.pth"
torch.save(model.state_dict(), MODEL_PATH)

print("✅ Model saved successfully at:")
print(MODEL_PATH)


✅ Model saved successfully at:
c:\Daily drive\College\sem 6\Main Project DL\unet_damage_model.pth
